In [1]:
import torch
import numpy as np
import pickle
import time
from pathlib import Path
from TCN.mnist_pixel.utils import data_generator, average_runtime, predict_image, model_accuracy
from TCN.mnist_pixel.model import TCN

from torch2trt import TRTModule

In [2]:
model_path = Path('./TCN/mnist_pixel/models')
trt_model_path = Path('./TCN/mnist_pixel/models_trt')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'aug_k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

_, test_loader = data_generator(data_path, batch_size)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Torch Model

In [3]:
# Load Model

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize, trt=True, apply_max=True)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')), strict=False)
model.eval()

if torch.cuda.is_available():
    model.cuda()

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=50, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='aug_k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1, seq_augment=True)


In [4]:
torch_accuracy = model_accuracy(model, test_loader, num_samples=100)
torch_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 0.9900990099009901
Accuracy after 150 images: 0.9801324503311258
Accuracy after 200 images: 0.9800995024875622
Accuracy after 250 images: 0.9800796812749004
Accuracy after 300 images: 0.9800664451827242
Accuracy after 350 images: 0.98005698005698
Accuracy after 400 images: 0.9825436408977556
Accuracy after 450 images: 0.9800443458980045
Accuracy after 500 images: 0.9820359281437125
Accuracy after 550 images: 0.9836660617059891
Accuracy after 600 images: 0.9800332778702163
Accuracy after 650 images: 0.978494623655914
Accuracy after 700 images: 0.9757489300998573
Accuracy after 750 images: 0.9747003994673769
Accuracy after 800 images: 0.9750312109862672
Accuracy after 850 images: 0.9753231492361927
Accuracy after 900 images: 0.97669256381798
Accuracy after 950 images: 0.9768664563617245
Accuracy after 1000 images: 0.977022977022977
Accuracy after 1050 images: 0.9752616555661275
Accuracy after 1100 image

Accuracy after 8850 images: 0.9781945542876511
Accuracy after 8900 images: 0.9783170430288731
Accuracy after 8950 images: 0.9784381633337057
Accuracy after 9000 images: 0.9784468392400845
Accuracy after 9050 images: 0.9781239642028505
Accuracy after 9100 images: 0.978244148994616
Accuracy after 9150 images: 0.9783630204349252
Accuracy after 9200 images: 0.9784805999347898
Accuracy after 9250 images: 0.9785969084423306
Accuracy after 9300 images: 0.9787119664552199
Accuracy after 9350 images: 0.9788257940327237
Accuracy after 9400 images: 0.9789384108073609
Accuracy after 9450 images: 0.9790498359961909
Accuracy after 9500 images: 0.9791600884117462
Accuracy after 9550 images: 0.9791644853941995
Accuracy after 9600 images: 0.9792729923966254
Accuracy after 9650 images: 0.9792767588850896
Accuracy after 9700 images: 0.9791774043912999
Accuracy after 9750 images: 0.9790790688134551
Accuracy after 9800 images: 0.9789817365574941
Accuracy after 9850 images: 0.978986904882753
Accuracy after 

0.9791

In [4]:
torch_accuracy_1_sample = model_accuracy(model, test_loader, num_samples=1)
torch_accuracy_1_sample

Accuracy after 0 images: 1.0
Accuracy after 50 images: 0.9607843137254902
Accuracy after 100 images: 0.9702970297029703
Accuracy after 150 images: 0.9602649006622517
Accuracy after 200 images: 0.9353233830845771
Accuracy after 250 images: 0.9362549800796812
Accuracy after 300 images: 0.9335548172757475
Accuracy after 350 images: 0.9316239316239316
Accuracy after 400 images: 0.940149625935162
Accuracy after 450 images: 0.9356984478935698
Accuracy after 500 images: 0.936127744510978
Accuracy after 550 images: 0.9401088929219601
Accuracy after 600 images: 0.9317803660565723
Accuracy after 650 images: 0.9262672811059908
Accuracy after 700 images: 0.9258202567760342
Accuracy after 750 images: 0.9267643142476698
Accuracy after 800 images: 0.9275905118601748
Accuracy after 850 images: 0.9306698002350177
Accuracy after 900 images: 0.93007769145394
Accuracy after 950 images: 0.9305993690851735
Accuracy after 1000 images: 0.9280719280719281
Accuracy after 1050 images: 0.9257849666983825
Accuracy

Accuracy after 8800 images: 0.9262583797295761
Accuracy after 8850 images: 0.9266749519828268
Accuracy after 8900 images: 0.9270868441748118
Accuracy after 8950 images: 0.9274941347335494
Accuracy after 9000 images: 0.9277858015776025
Accuracy after 9050 images: 0.9276323058225611
Accuracy after 9100 images: 0.9277002527194814
Accuracy after 9150 images: 0.928095290132226
Accuracy after 9200 images: 0.9280512987718726
Accuracy after 9250 images: 0.9283320722084099
Accuracy after 9300 images: 0.9287173422212666
Accuracy after 9350 images: 0.9288846112715218
Accuracy after 9400 images: 0.9291564727156686
Accuracy after 9450 images: 0.9295312665326421
Accuracy after 9500 images: 0.9299021155667825
Accuracy after 9550 images: 0.9300596796147
Accuracy after 9600 images: 0.9302156025414019
Accuracy after 9650 images: 0.9300590612371775
Accuracy after 9700 images: 0.9300072157509535
Accuracy after 9750 images: 0.9298533483745257
Accuracy after 9800 images: 0.9298030813182329
Accuracy after 98

0.9303

In [6]:
torch_accuracy_200_samples = model_accuracy(model, test_loader, num_samples=200)
torch_accuracy_200_samples

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 1.0
Accuracy after 150 images: 0.9933774834437086
Accuracy after 200 images: 0.9900497512437811
Accuracy after 250 images: 0.9880478087649402
Accuracy after 300 images: 0.9900332225913622
Accuracy after 350 images: 0.9886039886039886
Accuracy after 400 images: 0.9900249376558603
Accuracy after 450 images: 0.9866962305986696
Accuracy after 500 images: 0.9880239520958084
Accuracy after 550 images: 0.9891107078039928
Accuracy after 600 images: 0.9866888519134775
Accuracy after 650 images: 0.9861751152073732
Accuracy after 700 images: 0.9828815977175464
Accuracy after 750 images: 0.9813581890812251
Accuracy after 800 images: 0.982521847690387
Accuracy after 850 images: 0.982373678025852
Accuracy after 900 images: 0.9833518312985572
Accuracy after 950 images: 0.982124079915878
Accuracy after 1000 images: 0.983016983016983
Accuracy after 1050 images: 0.9819219790675547
Accuracy after 1100 images: 0.98274296

Accuracy after 8850 images: 0.9822618913117162
Accuracy after 8900 images: 0.9823615324120886
Accuracy after 8950 images: 0.9824600603284549
Accuracy after 9000 images: 0.9825574936118209
Accuracy after 9050 images: 0.9824328803447133
Accuracy after 9100 images: 0.9825293923744643
Accuracy after 9150 images: 0.9826248497431974
Accuracy after 9200 images: 0.9827192696446039
Accuracy after 9250 images: 0.9828126689006594
Accuracy after 9300 images: 0.9829050639716159
Accuracy after 9350 images: 0.9829964709656721
Accuracy after 9400 images: 0.9830869056483352
Accuracy after 9450 images: 0.9831763834514866
Accuracy after 9500 images: 0.9832649194821598
Accuracy after 9550 images: 0.9832478274526227
Accuracy after 9600 images: 0.9833350692636184
Accuracy after 9650 images: 0.9833177909024972
Accuracy after 9700 images: 0.983094526337491
Accuracy after 9750 images: 0.983078658599118
Accuracy after 9800 images: 0.9829609223548618
Accuracy after 9850 images: 0.9829458938178866
Accuracy after 

0.9831

# TensorRT Models

## torch2trt using ONNX

In [4]:
# Load Model
model_trt_onnx = TRTModule()
model_trt_onnx.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_onnx_amax.pt')))

<All keys matched successfully>

In [4]:
trt_onnx_accuracy = model_accuracy(model_trt_onnx, test_loader, num_samples=100)
trt_onnx_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 0.9900990099009901
Accuracy after 150 images: 0.9801324503311258
Accuracy after 200 images: 0.9800995024875622
Accuracy after 250 images: 0.9800796812749004
Accuracy after 300 images: 0.9800664451827242
Accuracy after 350 images: 0.98005698005698
Accuracy after 400 images: 0.9825436408977556
Accuracy after 450 images: 0.9800443458980045
Accuracy after 500 images: 0.9820359281437125
Accuracy after 550 images: 0.9836660617059891
Accuracy after 600 images: 0.9800332778702163
Accuracy after 650 images: 0.978494623655914
Accuracy after 700 images: 0.9757489300998573
Accuracy after 750 images: 0.9747003994673769
Accuracy after 800 images: 0.9750312109862672
Accuracy after 850 images: 0.9753231492361927
Accuracy after 900 images: 0.97669256381798
Accuracy after 950 images: 0.9768664563617245
Accuracy after 1000 images: 0.977022977022977
Accuracy after 1050 images: 0.9752616555661275
Accuracy after 1100 image

Accuracy after 8850 images: 0.9781945542876511
Accuracy after 8900 images: 0.9783170430288731
Accuracy after 8950 images: 0.9784381633337057
Accuracy after 9000 images: 0.9784468392400845
Accuracy after 9050 images: 0.9781239642028505
Accuracy after 9100 images: 0.978244148994616
Accuracy after 9150 images: 0.9783630204349252
Accuracy after 9200 images: 0.9784805999347898
Accuracy after 9250 images: 0.9785969084423306
Accuracy after 9300 images: 0.9787119664552199
Accuracy after 9350 images: 0.9788257940327237
Accuracy after 9400 images: 0.9789384108073609
Accuracy after 9450 images: 0.9790498359961909
Accuracy after 9500 images: 0.9791600884117462
Accuracy after 9550 images: 0.9791644853941995
Accuracy after 9600 images: 0.9792729923966254
Accuracy after 9650 images: 0.9792767588850896
Accuracy after 9700 images: 0.9791774043912999
Accuracy after 9750 images: 0.9790790688134551
Accuracy after 9800 images: 0.9789817365574941
Accuracy after 9850 images: 0.978986904882753
Accuracy after 

0.9791

In [6]:
trt_onnx_accuracy = model_accuracy(model_trt_onnx, test_loader, num_samples=50)
trt_onnx_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 0.9900990099009901
Accuracy after 150 images: 0.9801324503311258
Accuracy after 200 images: 0.9701492537313433
Accuracy after 250 images: 0.9721115537848606
Accuracy after 300 images: 0.973421926910299
Accuracy after 350 images: 0.9743589743589743
Accuracy after 400 images: 0.9775561097256857
Accuracy after 450 images: 0.9733924611973392
Accuracy after 500 images: 0.9760479041916168
Accuracy after 550 images: 0.9782214156079855
Accuracy after 600 images: 0.9700499168053245
Accuracy after 650 images: 0.967741935483871
Accuracy after 700 images: 0.9657631954350927
Accuracy after 750 images: 0.9653794940079894
Accuracy after 800 images: 0.9662921348314607
Accuracy after 850 images: 0.9670975323149236
Accuracy after 900 images: 0.9667036625971143
Accuracy after 950 images: 0.9674027339642481
Accuracy after 1000 images: 0.968031968031968
Accuracy after 1050 images: 0.966698382492864
Accuracy after 1100 ima

Accuracy after 8800 images: 0.9669355755027838
Accuracy after 8850 images: 0.967122359055474
Accuracy after 8900 images: 0.9673070441523425
Accuracy after 8950 images: 0.9674896659591107
Accuracy after 9000 images: 0.967559160093323
Accuracy after 9050 images: 0.9672964313335543
Accuracy after 9100 images: 0.9673662234919239
Accuracy after 9150 images: 0.9675445306523878
Accuracy after 9200 images: 0.9676122160634714
Accuracy after 9250 images: 0.9677872662414874
Accuracy after 9300 images: 0.9679604343618966
Accuracy after 9350 images: 0.9680248101807294
Accuracy after 9400 images: 0.9681948728858633
Accuracy after 9450 images: 0.9683631361760661
Accuracy after 9500 images: 0.9685296284601621
Accuracy after 9550 images: 0.9684849753952466
Accuracy after 9600 images: 0.9686490990521821
Accuracy after 9650 images: 0.9687079059164854
Accuracy after 9700 images: 0.9685599422739923
Accuracy after 9750 images: 0.9685160496359347
Accuracy after 9800 images: 0.9684726048362412
Accuracy after 

0.9686

In [7]:
trt_onnx_accuracy = model_accuracy(model_trt_onnx, test_loader, num_samples=150)
trt_onnx_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 1.0
Accuracy after 150 images: 0.9867549668874173
Accuracy after 200 images: 0.9850746268656716
Accuracy after 250 images: 0.9840637450199203
Accuracy after 300 images: 0.9867109634551495
Accuracy after 350 images: 0.9857549857549858
Accuracy after 400 images: 0.9875311720698254
Accuracy after 450 images: 0.9844789356984479
Accuracy after 500 images: 0.9860279441117764
Accuracy after 550 images: 0.9872958257713249
Accuracy after 600 images: 0.9833610648918469
Accuracy after 650 images: 0.9831029185867896
Accuracy after 700 images: 0.9800285306704708
Accuracy after 750 images: 0.9786950732356857
Accuracy after 800 images: 0.9800249687890137
Accuracy after 850 images: 0.9800235017626322
Accuracy after 900 images: 0.9811320754716981
Accuracy after 950 images: 0.9810725552050473
Accuracy after 1000 images: 0.9820179820179821
Accuracy after 1050 images: 0.9809705042816366
Accuracy after 1100 images: 0.9818

Accuracy after 8850 images: 0.9813580386397017
Accuracy after 8900 images: 0.9814627569935962
Accuracy after 8950 images: 0.9815663054407329
Accuracy after 9000 images: 0.9816687034773914
Accuracy after 9050 images: 0.9814385150812065
Accuracy after 9100 images: 0.9815404900560378
Accuracy after 9150 images: 0.9816413506720577
Accuracy after 9200 images: 0.9817411150961852
Accuracy after 9250 images: 0.9818398011025835
Accuracy after 9300 images: 0.9819374260832169
Accuracy after 9350 images: 0.9820340070580686
Accuracy after 9400 images: 0.9821295606850335
Accuracy after 9450 images: 0.9822241032694953
Accuracy after 9500 images: 0.9823176507736028
Accuracy after 9550 images: 0.9823055177468328
Accuracy after 9600 images: 0.9823976669096969
Accuracy after 9650 images: 0.9823852450523262
Accuracy after 9700 images: 0.9821667869291826
Accuracy after 9750 images: 0.9821556763408881
Accuracy after 9800 images: 0.9820426487093153
Accuracy after 9850 images: 0.9820322809867018
Accuracy afte

0.9822

## torch2trt using converters

In [5]:
# Load Model
model_trt_no_onnx = TRTModule()
model_trt_no_onnx.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_amax.pt')))

<All keys matched successfully>

In [6]:
trt_no_onnx_accuracy = model_accuracy(model_trt_no_onnx, test_loader, num_samples=100)
trt_no_onnx_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 0.9900990099009901
Accuracy after 150 images: 0.9801324503311258
Accuracy after 200 images: 0.9800995024875622
Accuracy after 250 images: 0.9800796812749004
Accuracy after 300 images: 0.9800664451827242
Accuracy after 350 images: 0.98005698005698
Accuracy after 400 images: 0.9825436408977556
Accuracy after 450 images: 0.9800443458980045
Accuracy after 500 images: 0.9820359281437125
Accuracy after 550 images: 0.9836660617059891
Accuracy after 600 images: 0.9800332778702163
Accuracy after 650 images: 0.978494623655914
Accuracy after 700 images: 0.9757489300998573
Accuracy after 750 images: 0.9747003994673769
Accuracy after 800 images: 0.9750312109862672
Accuracy after 850 images: 0.9753231492361927
Accuracy after 900 images: 0.97669256381798
Accuracy after 950 images: 0.9768664563617245
Accuracy after 1000 images: 0.977022977022977
Accuracy after 1050 images: 0.9752616555661275
Accuracy after 1100 image

Accuracy after 8850 images: 0.9781945542876511
Accuracy after 8900 images: 0.9783170430288731
Accuracy after 8950 images: 0.9784381633337057
Accuracy after 9000 images: 0.9784468392400845
Accuracy after 9050 images: 0.9781239642028505
Accuracy after 9100 images: 0.978244148994616
Accuracy after 9150 images: 0.9783630204349252
Accuracy after 9200 images: 0.9784805999347898
Accuracy after 9250 images: 0.9785969084423306
Accuracy after 9300 images: 0.9787119664552199
Accuracy after 9350 images: 0.9788257940327237
Accuracy after 9400 images: 0.9789384108073609
Accuracy after 9450 images: 0.9790498359961909
Accuracy after 9500 images: 0.9791600884117462
Accuracy after 9550 images: 0.9791644853941995
Accuracy after 9600 images: 0.9792729923966254
Accuracy after 9650 images: 0.9792767588850896
Accuracy after 9700 images: 0.9791774043912999
Accuracy after 9750 images: 0.9790790688134551
Accuracy after 9800 images: 0.9789817365574941
Accuracy after 9850 images: 0.978986904882753
Accuracy after 

0.9791

## torch2trt with ONNX - FP16

In [3]:
# Load Model
model_trt_onnx_fp16 = TRTModule()
model_trt_onnx_fp16.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_onnx_amax_fp16.pt')))

<All keys matched successfully>

In [4]:
trt_onnx_fp16_accuracy = model_accuracy(model_trt_onnx_fp16, test_loader, num_samples=100, fp16=True)
trt_onnx_fp16_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 0.9900990099009901
Accuracy after 150 images: 0.9801324503311258
Accuracy after 200 images: 0.9800995024875622
Accuracy after 250 images: 0.9800796812749004
Accuracy after 300 images: 0.9800664451827242
Accuracy after 350 images: 0.98005698005698
Accuracy after 400 images: 0.9825436408977556
Accuracy after 450 images: 0.9800443458980045
Accuracy after 500 images: 0.9820359281437125
Accuracy after 550 images: 0.9836660617059891
Accuracy after 600 images: 0.9800332778702163
Accuracy after 650 images: 0.978494623655914
Accuracy after 700 images: 0.9757489300998573
Accuracy after 750 images: 0.9747003994673769
Accuracy after 800 images: 0.9750312109862672
Accuracy after 850 images: 0.9753231492361927
Accuracy after 900 images: 0.97669256381798
Accuracy after 950 images: 0.9768664563617245
Accuracy after 1000 images: 0.977022977022977
Accuracy after 1050 images: 0.9752616555661275
Accuracy after 1100 image

Accuracy after 8850 images: 0.9780815727036493
Accuracy after 8900 images: 0.9782046961015616
Accuracy after 8950 images: 0.9783264439727405
Accuracy after 9000 images: 0.9783357404732808
Accuracy after 9050 images: 0.978013479173572
Accuracy after 9100 images: 0.9781342709592352
Accuracy after 9150 images: 0.978253742760354
Accuracy after 9200 images: 0.9783719160960765
Accuracy after 9250 images: 0.9784888120203221
Accuracy after 9300 images: 0.9786044511342866
Accuracy after 9350 images: 0.9787188535985456
Accuracy after 9400 images: 0.9788320391447718
Accuracy after 9450 images: 0.9789440270870807
Accuracy after 9500 images: 0.9790548363330176
Accuracy after 9550 images: 0.9790597843157784
Accuracy after 9600 images: 0.9791688365795229
Accuracy after 9650 images: 0.979173142679515
Accuracy after 9700 images: 0.9790743222348212
Accuracy after 9750 images: 0.9789765152292073
Accuracy after 9800 images: 0.9788797061524335
Accuracy after 9850 images: 0.9788853923459547
Accuracy after 9

0.979

## torch2trt with converters - FP16

In [5]:
# Load Model
model_trt_no_onnx_fp16 = TRTModule()
model_trt_no_onnx_fp16.load_state_dict(torch.load(trt_model_path / (model_name+'_trt_amax_fp16.pt')))

<All keys matched successfully>

In [6]:
trt_no_onnx_fp16_accuracy = model_accuracy(model_trt_no_onnx_fp16, test_loader, num_samples=100, fp16=True)
trt_no_onnx_fp16_accuracy

Accuracy after 0 images: 1.0
Accuracy after 50 images: 1.0
Accuracy after 100 images: 0.9900990099009901
Accuracy after 150 images: 0.9801324503311258
Accuracy after 200 images: 0.9800995024875622
Accuracy after 250 images: 0.9800796812749004
Accuracy after 300 images: 0.9800664451827242
Accuracy after 350 images: 0.98005698005698
Accuracy after 400 images: 0.9825436408977556
Accuracy after 450 images: 0.9800443458980045
Accuracy after 500 images: 0.9820359281437125
Accuracy after 550 images: 0.9836660617059891
Accuracy after 600 images: 0.9800332778702163
Accuracy after 650 images: 0.978494623655914
Accuracy after 700 images: 0.9757489300998573
Accuracy after 750 images: 0.9747003994673769
Accuracy after 800 images: 0.9750312109862672
Accuracy after 850 images: 0.9753231492361927
Accuracy after 900 images: 0.97669256381798
Accuracy after 950 images: 0.9768664563617245
Accuracy after 1000 images: 0.977022977022977
Accuracy after 1050 images: 0.9752616555661275
Accuracy after 1100 image

Accuracy after 8850 images: 0.9780815727036493
Accuracy after 8900 images: 0.9782046961015616
Accuracy after 8950 images: 0.9783264439727405
Accuracy after 9000 images: 0.9783357404732808
Accuracy after 9050 images: 0.978013479173572
Accuracy after 9100 images: 0.9781342709592352
Accuracy after 9150 images: 0.978253742760354
Accuracy after 9200 images: 0.9783719160960765
Accuracy after 9250 images: 0.9784888120203221
Accuracy after 9300 images: 0.9786044511342866
Accuracy after 9350 images: 0.9787188535985456
Accuracy after 9400 images: 0.9788320391447718
Accuracy after 9450 images: 0.9789440270870807
Accuracy after 9500 images: 0.9790548363330176
Accuracy after 9550 images: 0.9790597843157784
Accuracy after 9600 images: 0.9791688365795229
Accuracy after 9650 images: 0.979173142679515
Accuracy after 9700 images: 0.9790743222348212
Accuracy after 9750 images: 0.9789765152292073
Accuracy after 9800 images: 0.9788797061524335
Accuracy after 9850 images: 0.9788853923459547
Accuracy after 9

0.979

In [22]:
x = torch.ones(1,1,784).cuda() * 0.3
model_trt_no_onnx_fp16(x)


tensor([[0]], device='cuda:0', dtype=torch.int32)

In [11]:
import tensorrt as trt

In [17]:
b = trt.Builder(trt.Logger())

In [18]:
dir(b)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'average_find_iterations',
 'build_cuda_engine',
 'build_engine',
 'create_builder_config',
 'create_network',
 'create_optimization_profile',
 'debug_sync',
 'error_recorder',
 'fp16_mode',
 'gpu_allocator',
 'int8_calibrator',
 'int8_mode',
 'max_batch_size',
 'max_workspace_size',
 'min_find_iterations',
 'platform_has_fast_fp16',
 'platform_has_fast_int8',
 'platform_has_tf32',
 'refittable',
 'strict_type_constraints']

In [19]:
trt.__version__

'7.2.2.1'